<a href="https://colab.research.google.com/github/Vaibhavgoel123/Artificial-Intelligence/blob/Projects/Myproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

In [57]:
import numpy as np
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [73]:
import sys
sys.path.append("/content/drive/MyDrive")


## Core Idea
In this project Bidirectional LSTM and Attention model is used to focus on important points in any sentence so that the overall sentence can be classified into one of the 3 categories:  
1. Neutral
2. Negative
3. Positive

Bidirectional LSTM is used so that the sentence can be validated properly and Attention model is used so as to focus on important keywords like **good**, **bad**, **excellent** etc. to understand the user sentiments regarding to that video.

LSTM (Long Short Term Memory) is basically used to store or to understand the overall sentence while RNN (Recurrent Neural Networks) is only used for understanding the previous words or letters.

In [74]:
from model import Attention
from tensorflow.keras.layers import Bidirectional
# Dummy text
texts = [
    "I love this video",
    "This is terrible",
    "This is horrible",
    "This video is great",
    "Excellent presentation",
    "Amazing",
    "Good but not great"
]

labels=[2, 0, 0, 2, 2, 2, 1] #0-neg, 1-neutral, 2-positive
num_words = 20000
maxlen = 100
embedding_dim = 128

#Tokenizer
tokenizer = Tokenizer(num_words=num_words, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=maxlen)
y = to_categorical(labels, num_classes=3)

model = Sequential([
    Embedding(num_words, embedding_dim, input_length=maxlen, mask_zero=True),
    Bidirectional(LSTM(128, return_sequences=True)),
    Attention(),
    Dense(64, activation='relu'),
    Dropout(0.3), #Randomly deactivates 30% of the neurons
    Dense(3, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [75]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=5, batch_size=2)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'attention_5' (of type Attention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 282ms/step - accuracy: 0.4381 - loss: 1.0987
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - accuracy: 0.5286 - loss: 1.0952
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.5619 - loss: 1.0905
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.7119 - loss: 1.0840
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - accuracy: 0.6119 - loss: 1.0841


In [76]:
model.save('sentiment_model.keras')
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

print("Model and tokenizer saved")

Model and tokenizer saved


## Creating a Web API
Deployment of model is also necessary as its development so we are writing ```api.py``` file so as to connect this backend to the frontend in the form of API rather than as full code.

In [81]:
%%writefile api.py
# paste your FastAPI code here
import tensorflow as tf
import pickle
import numpy as np
from fastapi import FastAPI
from pydantic import BaseModel
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sys
sys.path.append("/content/drive/MyDrive")
from model import Attention

# The Attention class must be defined or imported here for load_model to work
# It's already defined in cell EvwvLbPgf44B and executed, so it's in scope.

class Comments(BaseModel):
  comments: list[str]

app = FastAPI()

# Provide custom_objects to load the Attention layer correctly
model = tf.keras.models.load_model("/content/drive/MyDrive/sentiment_model.keras", custom_objects={"Attention": Attention})
tokenizer = pickle.load(open("/content/drive/MyDrive/tokenizer.pkl", "rb"))

MAXLEN = 100
labels = ["negative", "neutral", "positive"]

@app.post("/predict_batch")
def predict_batch(data: Comments):
  seq = tokenizer.texts_to_sequences(data.comments)
  padded = pad_sequences(seq, maxlen=MAXLEN)

  preds = model.predict(padded)
  results = []

  for probs in preds:
    idx = int(np.argmax(probs))
    results.append(labels[idx])


  return {"results":results}

Overwriting api.py


In [90]:
!pip install fastapi uvicorn pyngrok nest_asyncio


Connecting the backend ```api.py``` to the browser to access the output.

In [89]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

ngrok.set_auth_token("YOUR_AUTH_TOKEN")
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(public_url)

!uvicorn api:app --host 0.0.0.0 --port 8000 &

NgrokTunnel: "https://nonnaturally-disjoinable-shakita.ngrok-free.dev" -> "http://localhost:8000"
2026-01-15 09:08:56.999219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768468137.036117   29881 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768468137.047654   29881 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768468137.073970   29881 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768468137.074024   29881 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more 

2026-01-15 09:09:05.767509: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'attention_5' (of type Attention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
INFO:     Started server process [29881]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     103.59.75.176:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.59.75.176:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.59.75.176:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.59.75.176:0 - "GET /openapi.json HTTP/1.1" 200 OK
/usr/local/lib/python3.12/dist-packages/keras/src/layers/la